# Double-Bracket Iteration Strategy: magnetic field (Ising model)
This notebook shows the diagonalization process of DBI using the magnetic field strategy, which varies the diagonal operator $D$ by gradient descent. To find the gradient with respect to $D$, parameterization of $D$ is required. For the purpose of this notebook, we represent it by the Ising model, i.e.

$$ D = \sum \alpha_i Z_i +\sum \beta_{ij}Z_iZ_j$$


The gradients are calculated under the premise that the diagonalization gain curve can be fitted by a polynomial, and that the iteration step duration is taken at the first dip of the curve.

In [ ]:
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from qibo import hamiltonians, set_backend, symbols
from qibo.hamiltonians import Hamiltonian, SymbolicHamiltonian
from qibo.quantum_info import random_hermitian
from qibo.models.dbi.double_bracket import DoubleBracketGeneratorType, DoubleBracketScheduling, DoubleBracketIteration
from qibo.models.dbi.utils import generate_pauli_operator_dict, decompose_into_pauli_basis, ParameterizationTypes
from qibo.models.dbi.utils_dbr_strategies import gradient_numerical, gradient_descent

In [ ]:
def visualize_matrix(matrix, title=""):
    """Visualize hamiltonian in a heatmap form."""
    fig, ax = plt.subplots(figsize=(5,5))
    ax.set_title(title)
    try:
        im = ax.imshow(np.absolute(matrix), cmap="inferno")
    except TypeError:
        im = ax.imshow(np.absolute(matrix.get()), cmap="inferno")
    fig.colorbar(im, ax=ax)


## Test on random Hamiltonian

In [ ]:
# backend
set_backend("numpy")
# initialize dbi object
nqubits = 5
h0 = random_hermitian(2**nqubits, seed=2)
scheduling = DoubleBracketScheduling.hyperopt
mode = DoubleBracketGeneratorType.single_commutator
n_taylor = 5
dbi = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0), scheduling=scheduling, mode=mode)
print("Initial off diagonal norm", dbi.off_diagonal_norm)
visualize_matrix(dbi.h.matrix, title=f'Random hamiltonian with L={nqubits}')

### Order 1: $D=\sum \alpha_iZ_i$

In [ ]:
# generate pauli_operator_dict
pauli_operator_dict = generate_pauli_operator_dict(nqubits=nqubits, parameterization_order=1)
d_coef = decompose_into_pauli_basis(dbi.h.matrix, list(pauli_operator_dict.values()))
d = sum([d_coef[i] * list(pauli_operator_dict.values())[i] for i in range(nqubits)])
grad = gradient_numerical(dbi, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)
print('The initial D coefficients:', d_coef)
print('Gradient:', grad)

In [ ]:
iters = 15
off_diagonal_norm_1, d_params_hist, s_step = gradient_descent(dbi, iters, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)

In [ ]:
plt.title(str(nqubits) + ' spins random hamiltonian')
plt.plot(off_diagonal_norm_1)
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

### Order 2: $D=\sum \alpha_iZ_i + \beta_{ij}Z_iZ_j$

In [ ]:
dbi = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0), scheduling=scheduling, mode=mode)

In [ ]:
# generate pauli_operator_dict
pauli_operator_dict = generate_pauli_operator_dict(nqubits=nqubits, parameterization_order=2)
d_coef = decompose_into_pauli_basis(dbi.h.matrix, list(pauli_operator_dict.values()))
d = sum([d_coef[i] * list(pauli_operator_dict.values())[i] for i in range(nqubits)])
grad = gradient_numerical(dbi, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)
print('The initial D coefficients:', d_coef)
print('Gradient:', grad)

In [ ]:
iters = 15
off_diagonal_norm_2, d_params_hist, s_step = gradient_descent(dbi, iters, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)

In [ ]:
plt.title(str(nqubits) + ' spins random hamiltonian')
plt.plot(off_diagonal_norm_1, label='order 1')
plt.plot(off_diagonal_norm_2, label='order 2')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

## Test on TFIM
Here we choose to customize our TFIM in the X axis using `SymbolicHamiltonian`. It is also possible to use Hadamard gate to rotate the TFIM inbuilt in `qibo`.

$$ H = -(\sum X_i X_{i+1} + \sum hZ_i)

In [ ]:
# generate the Hamiltonian
nqubits = 5
h = 1
H_TFIM = SymbolicHamiltonian( - h*symbols.Z(nqubits-1), nqubits=nqubits)
# add linear interaction terms
for i in range(nqubits-1):
    H_TFIM -= SymbolicHamiltonian(symbols.X(i)*symbols.X(i+1) + h*symbols.Z(i), nqubits=nqubits)
H_TFIM = H_TFIM.dense
visualize_matrix(H_TFIM.matrix, title=f'TFIM with L={nqubits} h={h}')

In [ ]:
# backend
set_backend("numpy")
# initialize dbi object
dbi_TFIM = DoubleBracketIteration(deepcopy(H_TFIM), scheduling=scheduling, mode=mode)

### Order 1: $D=\sum \alpha_iZ_i$

In [ ]:
dbi_TFIM_1 = DoubleBracketIteration(deepcopy(H_TFIM), scheduling=scheduling, mode=mode)
# generate pauli_operator_dict
pauli_operator_dict = generate_pauli_operator_dict(nqubits=nqubits, parameterization_order=1)
d_coef = decompose_into_pauli_basis(dbi_TFIM_1.h.matrix, list(pauli_operator_dict.values()))
d = sum([d_coef[i] * list(pauli_operator_dict.values())[i] for i in range(nqubits)])
grad = gradient_numerical(dbi_TFIM_1, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)
print('The initial D coefficients:', d_coef)
print('Gradient:', grad)

In [ ]:
NSTEPS = 3
off_diagonal_norm_1, d_params_hist, s_step = gradient_descent(dbi_TFIM_1, NSTEPS, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)

In [ ]:
plt.title(f'n={nqubits} h={h} TFIM, order=1')
plt.plot(off_diagonal_norm_1)
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

In [ ]:
# the final matrix
visualize_matrix(dbi_TFIM.h.matrix)

### Order 2

In [ ]:
dbi_TFIM_2 = DoubleBracketIteration(deepcopy(H_TFIM), scheduling=scheduling, mode=mode)
# generate pauli_operator_dict
pauli_operator_dict = generate_pauli_operator_dict(nqubits=nqubits, parameterization_order=2)
d_coef = decompose_into_pauli_basis(dbi_TFIM_2.h.matrix, list(pauli_operator_dict.values()))
d = sum([d_coef[i] * list(pauli_operator_dict.values())[i] for i in range(nqubits)])
grad = gradient_numerical(dbi_TFIM_2, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)
print('The initial D coefficients:', d_coef)
print('Gradient:', grad)

In [ ]:
off_diagonal_norm_2, d_params_hist, s_step = gradient_descent(dbi_TFIM_2, NSTEPS, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)

In [ ]:
plt.title(f'n={nqubits} h={h} TFIM')
plt.plot(off_diagonal_norm_1, label='order 1')
plt.plot(off_diagonal_norm_2, label='order 2')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

In conclusion, we see that the parameterization order or locality of the Pauli based parameterization for gradient descent does not affect significantly the effectiveness of double bracket diagonalization.